# Newton Solver Convergence

This notebook covers Newton-Raphson solver configuration and convergence monitoring in Pulsim.

## Contents
1. Newton Solver Options
2. Tolerances Configuration
3. Convergence History
4. DC Convergence Aids
5. Troubleshooting Convergence Issues

In [22]:
import pulsim as ps
import numpy as np
import matplotlib.pyplot as plt

## 1. Newton Solver Options

The `NewtonOptions` class controls the Newton-Raphson solver behavior.

In [23]:
# Create default Newton options
opts = ps.NewtonOptions()

print("Default NewtonOptions:")
print(f"  max_iterations:  {opts.max_iterations}")
print(f"  initial_damping: {opts.initial_damping}")
print(f"  min_damping:     {opts.min_damping}")
print(f"  auto_damping:    {opts.auto_damping}")
print(f"  track_history:   {opts.track_history}")

# Customize options
opts.max_iterations = 100
opts.initial_damping = 1.0  # Full Newton step

print(f"\nCustomized:")
print(f"  max_iterations: {opts.max_iterations}")

Default NewtonOptions:
  max_iterations:  50
  initial_damping: 1.0
  min_damping:     0.01
  auto_damping:    True
  track_history:   True

Customized:
  max_iterations: 100


## 2. Tolerances Configuration

The `Tolerances` class defines convergence criteria:

- **abstol**: Absolute tolerance (for small values)
- **reltol**: Relative tolerance (for large values)
- **vntol**: Voltage node tolerance
- **iatol**: Current absolute tolerance

In [24]:
# Create tolerances
tol = ps.Tolerances()

print("Default Tolerances:")
print(f"  voltage_abstol: {tol.voltage_abstol}")
print(f"  voltage_reltol: {tol.voltage_reltol}")
print(f"  current_abstol: {tol.current_abstol}")
print(f"  current_reltol: {tol.current_reltol}")
print(f"  residual_tol:   {tol.residual_tol}")

# Tighter tolerances for precision
tol_tight = ps.Tolerances()
tol_tight.voltage_abstol = 1e-12
tol_tight.voltage_reltol = 1e-6
tol_tight.current_abstol = 1e-15

print(f"\nTight Tolerances:")
print(f"  voltage_abstol: {tol_tight.voltage_abstol}")
print(f"  voltage_reltol: {tol_tight.voltage_reltol}")

Default Tolerances:
  voltage_abstol: 1e-09
  voltage_reltol: 0.001
  current_abstol: 1e-12
  current_reltol: 0.001
  residual_tol:   1e-09

Tight Tolerances:
  voltage_abstol: 1e-12
  voltage_reltol: 1e-06


## 3. Convergence History

The `ConvergenceHistory` class tracks iteration-by-iteration convergence progress.

In [25]:
# Create a circuit for convergence analysis
def build_test_circuit():
    ckt = ps.Circuit()
    gnd = ckt.ground()
    
    n1 = ckt.add_node("n1")
    n2 = ckt.add_node("n2")
    n3 = ckt.add_node("n3")
    
    ckt.add_voltage_source("V1", n1, gnd, 10.0)
    ckt.add_resistor("R1", n1, n2, 1000.0)
    ckt.add_resistor("R2", n2, n3, 2000.0)
    ckt.add_resistor("R3", n3, gnd, 3000.0)
    ckt.add_capacitor("C1", n2, gnd, 1e-6)
    
    return ckt

ckt = build_test_circuit()

# Run DC analysis and get convergence info
dc_result = ps.dc_operating_point(ckt)

print(f"DC Analysis Result:")
print(f"  Success: {dc_result.success}")
print(f"  Strategy used: {dc_result.strategy_used}")

# Access Newton result
newton_result = dc_result.newton_result
print(f"\nNewton Result:")
print(f"  Status: {newton_result.status}")
print(f"  Iterations: {newton_result.iterations}")
print(f"  Final residual: {newton_result.final_residual:.2e}")

DC Analysis Result:
  Success: True
  Strategy used: DCStrategy.Direct

Newton Result:
  Status: SolverStatus.Success
  Iterations: 2
  Final residual: 1.07e-18


In [26]:
# Create ConvergenceHistory object
history = ps.ConvergenceHistory()

print("ConvergenceHistory methods:")
print(f"  size(): {history.size()}")
print(f"  empty(): {history.empty()}")

# IterationRecord structure
record = ps.IterationRecord()
print(f"\nIterationRecord fields:")
print(f"  iteration: {record.iteration}")
print(f"  residual_norm: {record.residual_norm}")
print(f"  max_voltage_error: {record.max_voltage_error}")
print(f"  max_current_error: {record.max_current_error}")
print(f"  step_norm: {record.step_norm}")
print(f"  damping: {record.damping}")
print(f"  converged: {record.converged}")

ConvergenceHistory methods:
  size(): 0
  empty(): True

IterationRecord fields:
  iteration: 0
  residual_norm: 0.0
  max_voltage_error: 0.0
  max_current_error: 0.0
  step_norm: 0.0
  damping: 1.0
  converged: False


## 4. DC Convergence Aids

For difficult circuits, Pulsim provides convergence aids:

- **Gmin stepping**: Add small conductance to ground
- **Source stepping**: Gradually ramp source values
- **Pseudo-transient**: Add fictitious capacitors

In [27]:
# GminConfig
gmin_cfg = ps.GminConfig()
print("GminConfig:")
print(f"  initial_gmin:     {gmin_cfg.initial_gmin}")
print(f"  final_gmin:       {gmin_cfg.final_gmin}")
print(f"  reduction_factor: {gmin_cfg.reduction_factor}")
print(f"  max_steps:        {gmin_cfg.max_steps}")
print(f"  enable_logging:   {gmin_cfg.enable_logging}")

# Configure Gmin stepping
gmin_cfg.initial_gmin = 1e-3
gmin_cfg.final_gmin = 1e-12

GminConfig:
  initial_gmin:     0.01
  final_gmin:       1e-12
  reduction_factor: 10.0
  max_steps:        20
  enable_logging:   False


In [28]:
# SourceSteppingConfig
src_cfg = ps.SourceSteppingConfig()
print("SourceSteppingConfig:")
print(f"  initial_scale:  {src_cfg.initial_scale}")
print(f"  final_scale:    {src_cfg.final_scale}")
print(f"  initial_step:   {src_cfg.initial_step}")
print(f"  min_step:       {src_cfg.min_step}")
print(f"  max_step:       {src_cfg.max_step}")
print(f"  max_steps:      {src_cfg.max_steps}")
print(f"  max_failures:   {src_cfg.max_failures}")
print(f"  enable_logging: {src_cfg.enable_logging}")

SourceSteppingConfig:
  initial_scale:  0.0
  final_scale:    1.0
  initial_step:   0.25
  min_step:       0.01
  max_step:       0.5
  max_steps:      100
  max_failures:   5
  enable_logging: False


In [29]:
# PseudoTransientConfig
ptran_cfg = ps.PseudoTransientConfig()
print("PseudoTransientConfig:")
print(f"  initial_dt:            {ptran_cfg.initial_dt}")
print(f"  min_dt:                {ptran_cfg.min_dt}")
print(f"  max_dt:                {ptran_cfg.max_dt}")
print(f"  dt_increase:           {ptran_cfg.dt_increase}")
print(f"  dt_decrease:           {ptran_cfg.dt_decrease}")
print(f"  max_iterations:        {ptran_cfg.max_iterations}")
print(f"  convergence_threshold: {ptran_cfg.convergence_threshold}")
print(f"  enable_logging:        {ptran_cfg.enable_logging}")

PseudoTransientConfig:
  initial_dt:            1e-09
  min_dt:                1e-15
  max_dt:                1000.0
  dt_increase:           2.0
  dt_decrease:           0.5
  max_iterations:        1000
  convergence_threshold: 1e-06
  enable_logging:        False


In [30]:
# Complete DCConvergenceConfig
dc_cfg = ps.DCConvergenceConfig()

print("DCConvergenceConfig components:")
print(f"  strategy:              {dc_cfg.strategy}")
print(f"  max_strategy_attempts: {dc_cfg.max_strategy_attempts}")
print(f"  enable_random_restart: {dc_cfg.enable_random_restart}")
print(f"  gmin_config:           {type(dc_cfg.gmin_config).__name__}")
print(f"  source_config:         {type(dc_cfg.source_config).__name__}")
print(f"  pseudo_config:         {type(dc_cfg.pseudo_config).__name__}")
print(f"  init_config:           {type(dc_cfg.init_config).__name__}")

# Configure for difficult circuits
dc_cfg.gmin_config.initial_gmin = 1e-3
dc_cfg.source_config.max_steps = 50

DCConvergenceConfig components:
  strategy:              DCStrategy.Auto
  max_strategy_attempts: 3
  enable_random_restart: True
  gmin_config:           GminConfig
  source_config:         SourceSteppingConfig
  pseudo_config:         PseudoTransientConfig
  init_config:           InitializationConfig


## 5. Troubleshooting Convergence Issues

Common convergence problems and solutions:

In [31]:
# Per-variable convergence analysis
var_conv = ps.VariableConvergence()
print("VariableConvergence fields:")
print(f"  index:            {var_conv.index}")
print(f"  value:            {var_conv.value}")
print(f"  delta:            {var_conv.delta}")
print(f"  tolerance:        {var_conv.tolerance}")
print(f"  normalized_error: {var_conv.normalized_error}")
print(f"  converged:        {var_conv.converged}")
print(f"  is_voltage:       {var_conv.is_voltage}")

# PerVariableConvergence container
per_var = ps.PerVariableConvergence()
print(f"\nPerVariableConvergence:")
print(f"  size():              {per_var.size()}")
print(f"  empty():             {per_var.empty()}")
print(f"  all_converged():     {per_var.all_converged()}")
print(f"  non_converged_count: {per_var.non_converged_count()}")

VariableConvergence fields:
  index:            0
  value:            0.0
  delta:            0.0
  tolerance:        0.0
  normalized_error: 0.0
  converged:        False
  is_voltage:       True

PerVariableConvergence:
  size():              0
  empty():             True
  all_converged():     True
  non_converged_count: 0


### Convergence Troubleshooting Guide

| Problem | Symptom | Solution |
|---------|---------|----------|
| Stiff circuit | Slow convergence | Reduce timestep, use tighter tolerances |
| Floating nodes | NaN values | Check all nodes have DC path to ground |
| Large dynamic range | Oscillating residual | Enable Gmin stepping |
| Nonlinear devices | No convergence | Enable source stepping |
| Startup issues | First iteration fails | Use pseudo-transient continuation |

In [32]:
# Example: Checking solver status
print("SolverStatus values:")
for attr in dir(ps.SolverStatus):
    if not attr.startswith('_') and attr not in ['name', 'value']:
        status = getattr(ps.SolverStatus, attr)
        print(f"  {attr}: {ps.solver_status_to_string(status)}")

SolverStatus values:
  ConvergenceStall: ConvergenceStall
  Diverging: Diverging
  MaxIterationsReached: MaxIterationsReached
  NumericalError: NumericalError
  SingularMatrix: SingularMatrix
  Success: Success


## Summary

### Key Classes

| Class | Purpose |
|-------|---------|
| `NewtonOptions` | Solver iteration control |
| `Tolerances` | Convergence criteria |
| `IterationRecord` | Single iteration data |
| `ConvergenceHistory` | Full convergence trace |
| `DCConvergenceConfig` | Complete DC analysis config |
| `GminConfig` | Gmin stepping parameters |
| `SourceSteppingConfig` | Source ramping parameters |
| `PseudoTransientConfig` | Continuation method params |

### Example Usage

```python
# Configure DC analysis
dc_cfg = ps.DCConvergenceConfig()
dc_cfg.gmin_config.initial_gmin = 1e-3
dc_cfg.gmin_config.final_gmin = 1e-12
dc_cfg.source_config.max_steps = 50

# Tolerances (via NewtonOptions)
opts = ps.NewtonOptions()
opts.max_iterations = 100
opts.tolerances.voltage_abstol = 1e-12

# Run with custom config
result = ps.dc_operating_point(circuit, dc_cfg)

# Check result
if result.success:
    print(f"Converged in {result.newton_result.iterations} iterations")
else:
    print(f"Failed: {result.newton_result.status}")
```

**Next:** [Validation Framework](08_validation.ipynb)

In [33]:
import pulsim as ps

# Check export_benchmark functions signature
bench = ps.BenchmarkResult()

try:
    csv = ps.export_benchmark_csv(bench, "/tmp/test.csv")
    print("export_benchmark_csv(bench, path) works")
except Exception as e:
    print(f"Error: {e}")

try:
    csv = ps.export_benchmark_csv([bench])
    print(f"export_benchmark_csv([bench]) works, returns: {type(csv)}")
except Exception as e:
    print(f"Error: {e}")

Error: export_benchmark_csv(): incompatible function arguments. The following argument types are supported:
    1. (results: collections.abc.Sequence[pulsim._pulsim.BenchmarkResult]) -> str

Invoked with: <pulsim._pulsim.BenchmarkResult object at 0x1115a66b0>, '/tmp/test.csv'
export_benchmark_csv([bench]) works, returns: <class 'str'>
